In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())

print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available: True
Current device: 0
Device name: NVIDIA GeForce GTX 1650


In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

## 모델이 클래스 특성을 학습하기에 충분한 표본 갯수로 데이터 제거

> Machinery에서 데이터가 30개 이하인 클래스 수: 100
> 
> Assembly에서 데이터가 30개 이하인 클래스 수: 1583
>
> 제거 후, 남은 데이터: 13882, MACHINERY : 62 ASSEMBLY:209

In [4]:
data=pd.read_excel('filtered_dataset_30.xlsx')

In [5]:
print(len(data['Machinery'].unique()),len(data['Assembly'].unique()))

62 209


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13882 entries, 0 to 13881
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        13882 non-null  object 
 1   No.          13882 non-null  int64  
 2   Subject      13872 non-null  object 
 3   Machinery    13882 non-null  object 
 4   Assembly     13882 non-null  object 
 5   청구품목         13882 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    13881 non-null  object 
 8   Part No.2    2430 non-null   object 
 9   청구량          13818 non-null  float64
 10  견적           13698 non-null  object 
 11  견적수량         13818 non-null  float64
 12  견적화폐         13818 non-null  object 
 13  견적단가         13882 non-null  float64
 14  발주번호         13882 non-null  object 
 15  발주처          13882 non-null  object 
 16  발주           13882 non-null  object 
 17  발주수량         13818 non-null  float64
 18  발주금액         13818 non-null  float64
 19  D/T 

In [7]:
print(data['청구품목'].unique())

['GE POWER PACK FORK - E7(B)'
 'SAMSON SUPER STRONG DOUBLE BRAID ROPE 1 3/4", 300FT'
 'WIRE ROPE G)6X(S)19 A3 CMP SLPP 28MM X 400M' ... 'BRACKET '
 'WASHER, 10 ' 'COVER,MANIFOLD.EXH ']


### 전처리
1. 텍스트 클리닝
2. 결합 후 TF-IDF

> part.no.1 은 콤마 위치에 따른 세부적인 차이가 많은 텍스트이므로 특수기호 및 문자 유지 필요하다고 판단되어 별도 전처리 X

In [8]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)   
    text = re.sub(r'[^\w\s\*/\-\+.,#&]', '', text) 
    text = re.sub(r'\s+', ' ', text)    
    text = re.sub(r'\b(사용금지|사)\b', '', text, flags=re.IGNORECASE)    
    text = text.strip()    
    return text


In [9]:
# 청구품목 클리닝
data['cleaned_item'] = data['청구품목'].apply(preprocess_text)

> 청구품목 데이터에서는 각 단어 의미적 연관성보다 주요단어가 있는 것이므로, 가중치 부여하는 것으로 접근함

In [12]:
# 확인용
claim_items = data['청구품목'].tolist() 

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=30)
tfidf_matrix = tfidf.fit_transform(claim_items)

# 중요한 단어 추출
important_words = tfidf.get_feature_names_out()

print("청구품목 내 주요단어:", important_words)

청구품목 내 주요단어: ['as' 'bearing' 'bolt' 'charges' 'core' 'cover' 'cylinder' 'for' 'fuel'
 'gasket' 'gear' 'gp' 'head' 'hex' 'in' 'kit' 'nut' 'oil' 'plate' 'pump'
 'ring' 'screw' 'seal' 'sensor' 'set' 'shaft' 'spring' 'valve' 'washer'
 'water']


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 2. TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=30) 
tfidf_matrix = tfidf.fit_transform(data['cleaned_item'])

### 발주처 클리닝

> 부가단어 (CORPORATION, Corp, CO., Ltd, GmbH, Co., Inc, 주식회사, 상사, 공사, Co.,Ltd, Ltd, Pte Ltd, LLC) 제거

> 핵심 정보(회사명 직접 관련) emphasizing 함

In [14]:
def clean_supplier_name(name):
    name = name.lower()
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name)
    name = re.sub(r'\(사용금지\)', '', name)
    name = re.sub(r'u\.s\.a', '_usa', name)
    name = re.sub(r'\.', '', name)
    suffixes = r'(corporation|corp|company|co|incorporated|inc|limited|ltd|상사|공사|엔지니어링|주식회사|주|gmbh|pte ltd|llc)'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [24]:
suppliers = [
    'MATSUI(U.S.A) COROPRATION', 'KTI', '대광기업(주)', 'K.TH MARCO',
    'HAEIN Coporation_Cheonan', 'KOREA UCD CO.,LTD.', 'EAST WIND Gmbh', '인스알파',
    'ICON INTERNATIONAL, INC', '한국쉘석유㈜', 'EURO KYTEX ENGINEERING BV', '대동베아링상사',
    'MARINE HYDROTEC CO.,LTD.', '금안상사', 'TEST COMPANY',
    'PORT RELIEF ENGINEERING CO.,LTD.',
    'Caterpillar Marine Asia Pacific Pte Ltd', '(주)혜인',
    'SANWA COMMERCIAL CO.,LTD.', 'yusinHR Co., Ltd.', '(주)선진종합', 'FURUNO',
    'NISSIN REFRIGERATION  ENGINEERIN', '(주)우림공사',
    'HAEIN Coporation_Cheonan(사용금지)', 'KEMEL', 'REXNORD LLC-FALK MARINE GROUP',
    '유신에이치알(사용금지)', 'GEA KOREA LTD', '주안에너지㈜', 'SUNJIN ETECH Co.,Ltd.',
    '디에스알제강주식회사', '(주)한국에프에이디', 'ALBERT GMBH', 'Wartsila Korea Ltd.',
    '(주)선진엔텍(사용금지)', 'PIRIOU NAVAL', '(주)프러스엔지니어링', 'Taeyoung Enterprise',
    'SHINA', 'INS ALFA', 'KEMEL(KOMARINE)', '누리엔지니어링', 'RNK TECH CO.,LTD',
    'OS SYSTEM CO.,LTD', '씨코리아엔지니어링(주)', '(주)두원알앤에이', '합동듸젤사', '하이에어코리아(주)',
    'DESMI PUMPING TECHNOLOGY(SUZHOU) CO.,LTD', '한국마이콤',
    'HUMAN & ENGINEERING CO.,LTD'
]

cleaned_suppliers = [clean_supplier_name(supplier) for supplier in suppliers]
print(cleaned_suppliers)


['matsui_usa', 'kti', '대광기업', 'kth mar', 'haein _cheonan', 'korea ucd', 'east wind', '인스알파', 'in international', '한국쉘석유', 'euro kytex engineering bv', '대동베아링', 'marine hydrotec', '금안', 'test', 'port relief engineering', 'caterpillar marine asia pacific', '혜인', 'sanwa mmercial', 'yusinhr', '선진종합', 'furuno', 'nissin refrigeration engineerin', '우림', 'haein _cheonan', 'kemel', 'rexnord -falk marine group', '유신에이치알', 'gea korea', '안에너지', 'sunjin etech', '디에스알제강', '한국에프에이디', 'albert', 'wartsila korea', '선진엔텍', 'piriou naval', '프러스', 'taeyoung enterprise', 'shina', 'ins alfa', 'kemelkomarine', '누리', 'rnk tech', 'os system', '씨코리아', '두원알앤에이', '합동듸젤사', '하이에어코리아', 'desmi pumping technologysuzhou', '한국마이콤', 'human engineering']


In [16]:
#  HAEIN Corporation => HAEIN
def extract_important_part(name):
    if re.search(r'[가-힣]', name):
        name = re.sub(r'(기업|상사|종합|공사)', '', name)
        important_part = name.split()[0]  # 첫 단어 추출
    else:
        # 영문 이름의 경우 첫 번째 단어만 추출
        important_part = name.split()[0]
    
    return important_part

# 한번더 반복 HAEIN HAEIN Corporation
def emphasize_supplier_name(name):
    important_part = extract_important_part(name)
    emphasized_name = f"{important_part} {important_part} {name}"  # 중요한 부분을 반복
    return emphasized_name

In [17]:
suppliers = ['MATSUI(U.S.A) COROPRATION', 'taeyoung enterprise','HAEIN Coporation_Cheonan(사용금지)']
cleaned_suppliers = [clean_supplier_name(supplier) for supplier in suppliers]  # 전처리
emphasized_suppliers = [emphasize_supplier_name(supplier) for supplier in cleaned_suppliers]  # 강조

print(emphasized_suppliers)

['matsui_usa matsui_usa matsui_usa', 'taeyoung taeyoung taeyoung enterprise', 'haein haein haein _cheonan']


### 결합

In [25]:
# 청구품목 전처리 (TF-IDF 벡터화 적용)
data['cleaned_item'] = data['청구품목'].apply(preprocess_text)

# 파트 넘버 전처리 (별도 전처리 없음)
data['Part No.1'] = data['Part No.1'].astype(str)

# 발주처 전처리
data['cleaned_supplier'] = data['발주처'].apply(clean_supplier_name)
# data['emphasized_supplier'] = data['cleaned_supplier'].apply(emphasize_supplier_name)

# 4. 청구품목 + Part No.1 + 발주처 결합 (증강 없이)
data['combined_text'] = data['cleaned_item'].fillna('') + " " + data['Part No.1'].fillna('') + " " + data['cleaned_supplier'].fillna('')



In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(data['combined_text'])

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import torch

# 1. combined_text 벡터화 (청구품목 + Part No.1 + 발주처 결합된 텍스트)
tfidf = TfidfVectorizer(max_features=500)  # 필요에 따라 max_features 조정 가능
X = tfidf.fit_transform(data['combined_text'])  # combined_text 벡터화

# 2. 레이블 준비 
machinery_labels = data['Machinery'].values
assembly_labels = data['Assembly'].values

label_encoder_machinery = LabelEncoder()
y_machinery = label_encoder_machinery.fit_transform(machinery_labels)

label_encoder_assembly = LabelEncoder()
y_assembly = label_encoder_assembly.fit_transform(assembly_labels)

# 3. 스케일러 
# TF-IDF 벡터는 이미 스케일링된 값(0~1)

X_scaled = X.toarray()

# 4. Train-Test Split
X_train_val, X_test, y_train_val_machinery, y_test_machinery, y_train_val_assembly, y_test_assembly = train_test_split(
    X_scaled, y_machinery, y_assembly, test_size=0.2, random_state=42, stratify=y_machinery
)

X_train, X_val, y_train_machinery, y_val_machinery, y_train_assembly, y_val_assembly = train_test_split(
    X_train_val, y_train_val_machinery, y_train_val_assembly, test_size=0.2, random_state=42, stratify=y_train_val_machinery
)

# 5. CNN 모델용 텐서로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train_machinery, dtype=torch.long)
y_val_tensor = torch.tensor(y_val_machinery, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_machinery, dtype=torch.long)

# 6. XGBoost 모델용 NumPy 배열 준비 (이미 X_scaled는 NumPy 배열)
X_train_np = X_train
X_val_np = X_val
X_test_np = X_test

y_train_np = y_train_machinery
y_val_np = y_val_machinery
y_test_np = y_test_machinery

print(f"X_train_np shape: {X_train_np.shape}")
print(f"X_train_tensor shape: {X_train_tensor.shape}")

X_train_np shape: (8884, 500)
X_train_tensor shape: torch.Size([8884, 500])


### MLP

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

def build_mlp_model(input_shape, dropout_rate=0.3, learning_rate=0.0001):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),  # TF-IDF 입력 크기에 맞춤
        Dropout(dropout_rate),
        BatchNormalization(),
        
        Dense(256, activation='relu'),
        Dropout(dropout_rate),
        BatchNormalization(),
        
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        BatchNormalization(),
        
        Dense(64, activation='relu'),
        Dropout(dropout_rate),
        BatchNormalization(),
        
        Dense(32, activation='relu'),  # 추가된 레이어
        Dropout(dropout_rate),
        BatchNormalization(),
        
        Dense(62, activation='softmax')  # Machinery 클래스 수
    ])

    # 옵티마이저와 학습률 설정
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model


In [58]:
# 모델 학습 및 평가
def train_and_evaluate(dropout_rate, learning_rate, batch_size):
    model = build_mlp_model(dropout_rate=dropout_rate, learning_rate=learning_rate)
    
    # 콜백 설정
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

    # 모델 학습
    history = model.fit(
        X_train, y_train_machinery, 
        epochs=100, batch_size=batch_size, 
        validation_data=(X_val, y_val_machinery),  
        callbacks=[early_stopping, reduce_lr]
    )

    # 성능 평가
    accuracy = model.evaluate(X_test, y_test_machinery, verbose=0)
    print(f"Test Accuracy: {accuracy[1]:.4f}")
    
    return model, history



In [59]:
model, history = train_and_evaluate(dropout_rate=0.3, learning_rate=0.0005, batch_size=64)

Epoch 1/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0337 - loss: 4.2428 - val_accuracy: 0.0594 - val_loss: 4.0824 - learning_rate: 5.0000e-04
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3454 - loss: 3.1767 - val_accuracy: 0.3751 - val_loss: 3.2652 - learning_rate: 5.0000e-04
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4853 - loss: 2.4738 - val_accuracy: 0.5349 - val_loss: 2.2727 - learning_rate: 5.0000e-04
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5263 - loss: 2.1119 - val_accuracy: 0.5754 - val_loss: 1.7611 - learning_rate: 5.0000e-04
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5382 - loss: 1.9305 - val_accuracy: 0.6029 - val_loss: 1.5467 - learning_rate: 5.0000e-04
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5752 - loss: 1.7347 - val_accuracy: 0.6182 - val_loss: 1.4372 - learning_rate: 5.0000e-04
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc

In [65]:
def k_fold_cross_validation(X, y, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        
        model = build_mlp_model(input_shape=X_train.shape[1])
        
        history = model.fit(
            X_train, y_train,
            epochs=50, batch_size=64,
            validation_data=(X_val, y_val),
            verbose=0
        )
        
        val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(val_accuracy)
        print(f"Fold Accuracy: {val_accuracy}")

    avg_accuracy = np.mean(fold_accuracies)
    print(f"Average K-Fold Accuracy: {avg_accuracy}")

In [66]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

X = tfidf.fit_transform(data['combined_text']).toarray() 
y_machinery = label_encoder_machinery.fit_transform(data['Machinery'])
y_assembly = label_encoder_assembly.fit_transform(data['Assembly'])

# K-Fold 교차 검증 수행
k_fold_cross_validation(X, y_machinery, k=5)  

C:\Users\User\anaconda3\envs\ship\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Fold Accuracy: 0.7133597135543823
Fold Accuracy: 0.7320849895477295
Fold Accuracy: 0.7377521395683289
Fold Accuracy: 0.7172189950942993
Fold Accuracy: 0.7244236469268799
Average K-Fold Accuracy: 0.724967896938324


In [68]:
from xgboost import XGBClassifier

machinery_optimized_model = XGBClassifier(
    objective='multi:softmax',
    num_class=62,
    learning_rate=0.0746,
    max_depth=10,
    n_estimators=148,
    subsample=1.0,
    colsample_bytree=0.6804,
    reg_lambda=1,
    random_state=42,
    verbosity=1
)

machinery_optimized_model.fit(X_train, y_train_machinery)

from sklearn.metrics import accuracy_score

y_pred_machinery = machinery_optimized_model.predict(X_test)
accuracy = accuracy_score(y_test_machinery, y_pred_machinery)
print(f"Accuracy of the optimized model: {accuracy:.4f}")

Accuracy of the optimized model: 0.7515
